In [41]:
import os
import subprocess
from Bio.Seq import Seq
import pandas as pd
#Off-target junctions
#L02: tattagccgcttacggaatg
#L09: aaggcggatagcgatacaga
#L14: gttaaggtcaaaccagttta
#L21: cacaatacgattaaattaat

In [96]:
#miseq mini reference generator
#first read files of just between the genome primers then read both LR and RL insert direction files
#crRNA's
#L02: accagacccgcgagcattaattcttgcctcca
#L09: tggtggcggtggtgggagctattctcgttctg
#L14: gaaaacacctgatatgaaaggcaatgccacca
#L21: agctgcaacaatgttgaaaatgccagccaact
path = 'sequencing-refs/L02/'
genome_file = open(path + "L02.fa", "r")
first = genome_file.readline()
genome_str = genome_file.read()
genome_file.close()
genome_str = genome_str.replace('\n','')

RL_file = open(path + "../R-sc-GFP-L.fa", "r")
first = RL_file.readline()
RL_str = RL_file.read()
RL_file.close()
RL_str = RL_str.replace('\n','')


LR_file = open(path + "../L-GFP-sc-R.fa", "r")
first = LR_file.readline()
LR_str = LR_file.read()
LR_file.close()
LR_str = LR_str.replace('\n','')

crRNA = 'accagacccgcgagcattaattcttgcctcca'
crRNA_ind = genome_str.index(crRNA) + 32
crRNA_ind

153

In [97]:
# start at 40th index after crRNA_ind and end at 61th
counter = 40
seed = []
headers = []
seed_window = 16
while counter <= 61:
    seed_1 = genome_str[crRNA_ind + counter - int(seed_window/2)-1: crRNA_ind + counter-1].upper() + RL_str[:8].upper()
    seed_2 = genome_str[crRNA_ind + counter - int(seed_window/2)-1: crRNA_ind + counter-1].upper() + LR_str[:8].upper()
    seed.append(seed_1)
    seed.append(seed_2)
    headers.append(f'>{counter}bp-RL-direction')
    headers.append(f'>{counter}bp-LR-direction')
    counter += 1
seed[:-2]
headers[:-2]
seed_with_headers = {}
for i in range(len(seed)):
    seed_with_headers.update({headers[i]: seed[i]})

In [98]:
path = 'sequencing-refs/L02-off-target/'
with open(path+ 'off-tar-' +"L02.fastq") as file:
    lines = [line.rstrip() for line in file]
line_index = [x for x in range(len(lines)) if "@" in lines[x]]
reads = []
for i in line_index:
    reads.append(lines[i+1])
with open(path + 'reads.txt', 'w') as f:
    for x in reads:
        f.write("%s\n" % x)
reads_file = path + 'reads.txt'

In [99]:
off_tar_seed = 'tattagccgcttacggaatg'.upper()
off_tar_output53 = path + 'off-tar-output53.txt'
off_tar_output35 = path + 'off-tar-output35.txt'
os.remove(off_tar_output53)
os.remove(off_tar_output35)
seq = Seq(off_tar_seed)
os.system(f'grep {off_tar_seed} {reads_file} | wc -l >> {off_tar_output53}')
os.system(f'grep {seq.reverse_complement()} {reads_file} | wc -l >> {off_tar_output35}')

0

In [100]:
output53 = path + 'output53.txt'
output35 = path + 'output35.txt'
os.remove(output53)
os.remove(output35)
for x in seed_with_headers.values():
    seq = Seq(x)
    os.system(f'grep {x} {reads_file} | wc -l >> {output53}')
    os.system(f'grep {seq.reverse_complement()} {reads_file} | wc -l >> {output35}')

In [38]:
path = 'sequencing-refs/L21-off-target/'
output53 = path + 'output53.txt'
output35 = path + 'output35.txt'
off_tar_output53 = path + 'off-tar-output53.txt'
off_tar_output35 = path + 'off-tar-output35.txt' 
o53 = open(output53, 'r')
o35 = open(output53, 'r')
ot_o53 = open(off_tar_output53, 'r')
ot_o35 = open(off_tar_output35, 'r')
o53_lines = [int(line.rstrip()) for line in o53]
o35_lines = [int(line.rstrip()) for line in o35]
ot_o53_lines = [int(line.rstrip()) for line in ot_o53]
ot_o35_lines = [int(line.rstrip()) for line in ot_o35]
on_target_sum = sum(o53_lines) + sum(o35_lines)
off_target_sum = sum(ot_o53_lines) + sum(ot_o35_lines)

In [30]:
df = pd.DataFrame(columns = ['loci', 'proportion', 'on or off'])

In [39]:
off_tar = off_target_sum/(on_target_sum + off_target_sum)
on_tar = on_target_sum/(on_target_sum + off_target_sum)
df.loc[len(df)] = ['L21', on_tar*100, 'On-Target']
df.loc[len(df)] = ['L21', off_tar*100, 'Off-Target']

In [43]:
df.to_csv(path + '../on-vs-off-target.csv')
df

,loci,proportion,on or off
0,L02,97.314801,On-Target
1,L02,2.685199,Off-Target
2,L09,98.853060,On-Target
3,L09,1.146940,Off-Target
4,L14,98.251113,On-Target
5,L14,1.748887,Off-Target
6,L21,98.283996,On-Target
7,L21,1.716004,Off-Target
